
## Mathematical Perspective

### 1. Policy Gradient Theorem

The policy gradient theorem provides the foundation for updating the policy. The objective is to maximize the expected cumulative reward:

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left[ \sum_{t=0}^{T} \gamma^t r_t \right],
$$

where  
• $ \tau $ is a trajectory sampled from the policy $\pi_\theta$,  
• $ \gamma \in (0,1) $ is the discount factor,  
• $ r_t $ is the reward at time $ t $.

The gradient update is:

$$
\nabla_\theta J(\theta) \approx \mathbb{E} \left[ \nabla_\theta \log \pi_\theta(a|s) \, A(s, a) \right],
$$

where $A(s, a)$ is the advantage function.

---

### 2. Generalized Advantage Estimation (GAE)

GAE reduces variance by considering a weighted sum of temporal differences. The temporal difference (TD) error is:

$$
\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t).
$$

Then, the advantage estimate $ A_t $ is computed as:

$$
A_t = \delta_t + \gamma \lambda \, \delta_{t+1} + \gamma^2 \lambda^2 \, \delta_{t+2} + \dots,
$$

or equivalently in a recursive form:

$$
A_t = \delta_t + \gamma \lambda A_{t+1}.
$$

In the code, this is implemented by iterating backward over the rewards, where $ \lambda $ is the GAE parameter (sometimes set to 0.95).

---

### 3. Clipped Surrogate Objective

PPO uses a clipped surrogate objective to constrain the update and ensure that the new policy does not deviate drastically from the old one. Define the probability ratio:

$$
r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_\text{old}}(a_t|s_t)}.
$$

The clipped objective function is:

$$
L^{\text{CLIP}}(\theta) = \mathbb{E} \left[ \min \left( r_t(\theta) A_t, \, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t \right) \right],
$$

where $ \epsilon $ is a small hyperparameter (e.g., 0.2). Here, the clipping limits the change in $ r_t $ to prevent large policy updates.

---

### 4. Critic Loss

The value function is optimized by minimizing the mean squared error between the predicted value $ V(s) $ and the computed return $ R_t $:

$$
L^{\text{VF}}(\theta_v) = \mathbb{E} \left[ \left( V(s_t) - R_t \right)^2 \right].
$$

---

## Programming Perspective

### Structure of the PPOAgent

1. **Initialization:**

   The agent is initialized with:
   - A Gymnasium environment.
   - Two neural networks, one for the policy and one for the value function.
   - Relevant hyperparameters such as:
     - `gae_lambda` (used in the GAE formula),
     - `clip_epsilon` (used in the surrogate objective),
     - `update_epochs`, `batch_size`, and `gamma`.

2. **Action Sampling and Evaluation:**

   - **Sampling:**  
     The `sample_actions` method samples actions from a distribution returned by the policy network. It also computes the log probability, which is needed later to compute the ratio $ r_t(\theta) $:
     
     ```python
     distr = self.policy_net.get_distribution(states)
     actions = distr.sample()
     log_probs = distr.log_prob(actions)
     ```

   - **Evaluation:**  
     The `eval_actions` method selects actions in a deterministic manner, typically using the mode of the distribution for evaluation.

3. **Trajectory Collection:**

   The `_collect_dataset` method gathers experience by interacting with the environment. For each step, it records:
   - State, action, reward, log probability, value estimate, and whether the state is terminal.
   
   After collecting the trajectory, it calls `_compute_gae_and_returns` to compute the returns and the advantages using the formulas discussed above.

4. **Computing GAE and Returns:**

   In `_compute_gae_and_returns`, the returns are calculated as:

   $$
   R_t = r_t + \gamma R_{t+1} \cdot (1 - d_t),
   $$

   where $ d_t $ is a done flag (0 or 1).  
   Similarly, the advantage is computed recursively as:

   $$
   A_t = \delta_t + \gamma \lambda A_{t+1} \cdot (1 - d_t),
   $$

   where

   $$
   \delta_t = r_t + \gamma V(s_{t+1}) - V(s_t).
   $$

5. **Policy (Actor) and Value (Critic) Updates:**

   - **Actor Update:**  
     The policy network is updated based on the negative of the clipped surrogate objective. The ratio $ r_t $ and the clip mechanism are used to guarantee a stable update:
     
     ```python
     ratio = torch.exp(new_batch_log_probs - batch_log_probs)
     clipped_ratio = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon)
     surr1 = ratio * batch_advantages
     surr2 = clipped_ratio * batch_advantages
     actor_loss = -torch.min(surr1, surr2).mean()
     ```

   - **Critic Update:**  
     The value network is updated by minimizing the mean squared error between the computed returns and the predicted state values:

     ```python
     critic_loss = (new_batch_values - batch_returns).pow(2).mean()
     ```

6. **Saving and Loading Models:**

   The agent provides methods to save and load the weights of the policy and value networks, ensuring the model can be persisted and later restored for inference or continued training.

---

## Summary

- **Mathematically:**  
  PPO uses policy gradients enhanced by Generalized Advantage Estimation (GAE) to compute low-variance advantage estimates. The core update is driven by a clipped surrogate objective that limits the difference between the old and new policies, ensuring stable training. Additionally, the critic is trained via a regression loss on the calculated returns.

- **Programmatically:**  
  The implementation within the `PPOAgent` class mirrors these ideas:
  - It samples actions, collects trajectories, computes the required returns and advantages using backward recursion, and updates both the actor and critic via gradients computed on mini-batches.
  - The clear separation of concerns (data collection, advantage/return computation, and network updates) translates the mathematical formulation into an organized and modular code structure.




## File: common.py

This file defines a generic multi-layer perceptron (MLP) which can be used as a building block to create both policy and value networks for reinforcement learning agents.

### What the Code Does

- **Imports:**  
  The file imports PyTorch’s neural network module (`torch.nn as nn`) which provides the building blocks for deep learning.

- **MLP Class:**  
  The `MLP` class inherits from `nn.Module`, making it a standard PyTorch module. This allows it to be integrated into larger networks and trained using PyTorch’s autograd system.

- **Constructor (__init__):**  
  The constructor accepts:
  - `in_dim`: The input dimension.
  - `out_dim`: The output dimension.
  - `hidden_dims`: A list of integers specifying the sizes of the hidden layers.
  
  The constructor builds the architecture dynamically:
  - It creates a list `hidden_dims` that starts with the input dimension and ends with the output dimension, with any intermediate hidden sizes in between.
  - It then iterates through pairs of dimensions (for each layer) and adds a `Linear` layer followed by a `ReLU` activation for all but the last layer.
  - Finally, the layers are packed into an `nn.Sequential` module assigned to `self.net`.

- **Forward Method:**  
  The forward method defines how the input data flows through the network:
  - It simply passes the input `x` through the sequential network (`self.net`).

### How It Works

1. **Layer Construction:**  
   Given an input and output dimension along with any hidden layers, the class builds a fully-connected network. For example, if `in_dim=4`, `hidden_dims=[64, 64]`, and `out_dim=2`, then `hidden_dims` becomes `[4, 64, 64, 2]`.  
   - A linear layer maps from 4 -> 64, followed by a ReLU.
   - Then a linear layer from 64 -> 64, followed by a ReLU.
   - Finally, a linear layer mapping 64 -> 2.

2. **Activation Functions:**  
   The `ReLU` activations introduce non-linearity, which is essential for the network to learn complex functions.

3. **Sequential Module:**  
   Packing the layers into `nn.Sequential` makes the forward pass concise since it simply applies all layers in order.

### Where It Is Used

- **Policy and Value Networks:**  
  In reinforcement learning projects like RL-2025, policy and value networks are the key components of many algorithms such as PPO.  
  - The `MLP` defined in common.py can be used to create the neural network architectures for both the policy network (which decides actions) and the value network (which estimates state values).
  - For example, in the PPOAgent (as seen in your project), instances of policy_net and value_net could be created using `MLP` for their respective architectures.

- **Reusability:**  
  By defining a generic MLP in a common file, the project promotes code reuse. Other parts of the project that require simple feed-forward networks can import and utilize this class—reducing code duplication and ensuring consistency.

- **Modularity:**  
  Placing network definitions in a separate module (like this one) keeps the project structure clean. It decouples the network architecture design from the agent's logic, making each part easier to maintain and update.

---

## Summary

- **Purpose:**  
  The file defines a flexible, configurable MLP useful for constructing any fully-connected layer-based network.

- **Usage:**  
  It is used by agents (e.g., PPOAgent) to create policy and value networks that are required for tasks like action selection, computing value estimates, and policy updates.

- **Design:**  
  The design leverages PyTorch’s `nn.Sequential` to dynamically build the network from the given dimensions, applying linear transformations and non-linear ReLU activations appropriately.




## 1. Policy Gradient and the Surrogate Objective

In on-policy methods, we want to maximize the expected cumulative reward:

$$
J(\theta)=\mathbb{E}_{\tau\sim\pi_\theta}\left[\sum_{t=0}^T \gamma^t r_t\right],
$$

where $ \tau=(s_0,a_0,s_1,\dots) $ is a trajectory sampled according to the policy $ \pi_\theta $, $ \gamma\in(0,1) $ is the discount factor, and $ r_t $ is the reward at step $ t $.

Using the policy gradient theorem, the gradient of the expected return is approximately:

$$
\nabla_\theta J(\theta) \approx \mathbb{E}\left[ \nabla_\theta \log \pi_\theta(a_t|s_t) \, A_t \right],
$$

where $ A_t = Q(s_t,a_t) - V(s_t) $ is the advantage, estimated from the collected trajectory.

---

## 2. Generalized Advantage Estimation (GAE)

To obtain a low-variance, low-bias estimator for the advantage $ A_t $, PPO employs Generalized Advantage Estimation (GAE). The temporal-difference error (TD error) is defined as:

$$
\delta_t = r_t + \gamma\, V(s_{t+1}) - V(s_t).
$$

GAE then computes the advantage recursively as:

$$
A_t = \delta_t + \gamma\lambda\,A_{t+1},
$$

or equivalently, as the weighted sum:

$$
A_t = \sum_{l=0}^{\infty} (\gamma\lambda)^l\, \delta_{t+l},
$$

where $ \lambda \in [0,1] $ is a parameter that tunes the bias–variance trade-off. In the code, this recursion is implemented in the `_compute_gae_and_returns` method.

---

## 3. Clipped Surrogate Objective

PPO improves stability by constraining the update step. Instead of performing a full policy gradient update, it maximizes a “clipped” surrogate objective. Define the probability ratio:

$$
r_t(\theta) = \frac{\pi_\theta (a_t|s_t)}{\pi_{\theta_\text{old}}(a_t|s_t)},
$$

and then the clipped objective is given by:

$$
L^{\text{CLIP}}(\theta)=\mathbb{E}\Bigl[\min\Bigl(r_t(\theta)\,A_t,\,\text{clip}\bigl(r_t(\theta),1-\epsilon,1+\epsilon\bigr)A_t\Bigr)\Bigr],
$$

where $ \epsilon $ (e.g., 0.2 in the code) is a hyperparameter determining the clipping range. This clipping prevents large updates by ensuring that $ r_t $ remains within $ [1-\epsilon,1+\epsilon] $. In the code, you see this implemented in the `_update_actor` method:

- The ratio is computed as:

  $$
  \texttt{ratio} = \exp(\texttt{new\_batch\_log\_probs} - \texttt{batch\_log\_probs}).
  $$

- The clipped ratio is then:

  $$
  \texttt{clipped\_ratio} = \texttt{torch.clamp(ratio, 1-\texttt{clip\_epsilon}, 1+\texttt{clip\_epsilon})}.
  $$

- The surrogate objectives are calculated as:

  $$
  \texttt{surr1} = \texttt{ratio} \times \texttt{batch\_advantages} \quad \text{and} \quad \texttt{surr2} = \texttt{clipped\_ratio} \times \texttt{batch\_advantages}.
  $$

- The actor loss then becomes:

  $$
  \texttt{actor\_loss} = -\texttt{torch.min(surr1, surr2).mean()},
  $$

which is exactly the negative of the clipped surrogate objective, ready to be minimized by gradient descent.

---

## 4. Value Function Loss

The critic (value network) approximates the state value $ V(s) $ and is updated by minimizing the Mean Squared Error (MSE) between the predicted value and the calculated return $ R_t $:

$$
L^{\text{VF}}(\theta_v)=\mathbb{E}\left[\bigl(V_{\theta_v}(s_t)-R_t\bigr)^2\right].
$$

In the code, this is implemented in the `_update_critic` method:

- The loss is computed as:

  $$
  \texttt{critic\_loss} = \bigl(\texttt{new\_batch\_values} - \texttt{batch\_returns}\bigr)^2.\texttt{mean()},
  $$

and then backpropagated using the optimizer of the value network.

---

## 5. Trajectory Collection

The agent collects trajectories by interacting with the environment. For each step, the following are recorded:

- The current state $ s_t $,
- The selected action $ a_t $ sampled from the policy,
- The reward $ r_t $,
- The log probability $ \log \pi(a_t|s_t) $,
- The value $ V(s_t) $,
- The done flag indicating the end of an episode.

Once a trajectory is complete, returns $ R_t $ and advantages $ A_t $ are computed using the formulas from GAE and discounted returns:

$$
R_t = r_t + \gamma\, R_{t+1}\,(1-d_t),
$$

where $ d_t $ denotes whether the state was terminal.

The conversion to tensors allows these values to be processed in mini-batches during the update loops.

---

## 6. Summary in the Context of the Code

- The **`_collect_dataset`** method gathers trajectories and then uses **`_compute_gae_and_returns`** to compute $ R_t $ and $ A_t $ using recursion.
- The agent’s **`train_step`** performs multiple epochs of update:
  - It splits the trajectories into batches,
  - Re-computes the new log probabilities and state values,
  - And updates the policy (actor) using the clipped surrogate objective as well as the value network (critic) using MSE loss.
- The clipping in the objective ensures that the policy does not deviate abruptly from the previous policy, enhancing training stability.




### 1. Collecting Trajectories

First, the agent interacts with the environment to collect trajectories. For each time step $ t $, the following are recorded:
- **State:** $ s_t $
- **Action:** $ a_t $ sampled from the current policy $ \pi_{\theta} $
- **Reward:** $ r_t $
- **Log-Probability:** $ \log \pi_{\theta}(a_t|s_t) $
- **Value:** $ V(s_t) $ from the critic  
- **Done Flag:** $ d_t $ indicating episode termination

This forms a sequence:
$$
\{s_t, a_t, r_t, \log \pi(a_t|s_t), V(s_t), d_t\}_{t=0}^{T}
$$

---

### 2. Generalized Advantage Estimation (GAE)

The algorithm then computes the advantage for each time step using GAE. For each step $ t $, the temporal-difference error is:
$$
\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)
$$
Then, the advantage $ A_t $ is computed recursively as:
$$
A_t = \delta_t + \gamma \lambda \, A_{t+1} \, (1-d_t)
$$
where:
- $ \gamma $ is the discount factor.
- $ \lambda $ is the GAE parameter that trades off bias and variance.

The algorithm also computes the returns:
$$
R_t = r_t + \gamma R_{t+1} \, (1-d_t)
$$

Both sequences $ \{A_t\} $ and $ \{R_t\} $ are used in the updates.

---

### 3. The PPO Surrogate Objective

PPO updates the policy by optimizing a clipped surrogate objective. First, the probability ratio is defined as:
$$
r_t(\theta) = \frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}
$$
In code, since log probabilities are computed, this ratio is evaluated as:
$$
r_t = \exp\left[\log\pi_{\theta}(a_t|s_t) - \log\pi_{\theta_{\text{old}}}(a_t|s_t)\right]
$$
The surrogate objective is then given by:
$$
L^{\text{CLIP}}(\theta) = \mathbb{E}_t \left[ \min\left( r_t(\theta) A_t, \, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \, A_t \right) \right]
$$
Here:
- $\epsilon$ is the clip parameter (e.g., 0.2) that restricts how far the new policy can deviate from the old policy.
- The minimum operator chooses the clipped or unclipped value to discourage updates that would move $ r_t $ far from 1.

The actor (policy) loss in code is the negative of this expectation:
$$
\text{actor\_loss} = -\frac{1}{N} \sum_{t} \min\left( r_t(\theta) A_t, \, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) \, A_t \right)
$$

---

### 4. Critic Update

The critic (value function) is updated by minimizing the mean squared error (MSE) between its value predictions $ V(s_t) $ and the computed returns $ R_t $:
$$
L_{\text{critic}}(\theta_v) = \frac{1}{N} \sum_{t} \left( V(s_t) - R_t \right)^2
$$
This loss function helps the critic learn accurate value estimates, which in turn improve the calculation of advantages.

---

### 5. Optimization Steps

The algorithm then performs several epochs of updates over mini-batches:
- **Policy Update:**  
  For each mini-batch, compute the new log probabilities $ \log\pi_{\theta}(a_t|s_t) $ and the ratio $ r_t $. Use the surrogate objective to compute actor loss, backpropagate, and update the policy network.
  
- **Value Update:**  
  For the same mini-batch, compute the critic’s predicted values and then the MSE loss. Backpropagate and update the value network.

---

### 6. Summary of the Mathematical Flow

1. **Data Collection:**  
   Collect trajectories $\{s_t, a_t, r_t, \log\pi_{\theta}(a_t|s_t), V(s_t), d_t\}$.

2. **Advantage & Return Computation:**  
   $$
   \delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)
   $$
   $$
   A_t = \delta_t + \gamma\lambda A_{t+1}(1-d_t)
   $$
   $$
   R_t = r_t + \gamma R_{t+1}(1-d_t)
   $$

3. **Policy Update using Clipping:**
   $$
   r_t(\theta) = \frac{\pi_{\theta}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} = \exp\left[\log\pi_{\theta}(a_t|s_t) - \log\pi_{\theta_{\text{old}}}(a_t|s_t)\right]
   $$
   $$
   L^{\text{CLIP}}(\theta) = \min \left( r_t(\theta) A_t, \, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t \right)
   $$
   Minimize the negative of the expected surrogate objective over mini-batches.

4. **Critic Update:**  
   Minimize the MSE:
   $$
   L_{\text{critic}} = \frac{1}{N}\sum_t \left( V(s_t) - R_t \right)^2
   $$

5. **Iterative Optimization:**  
   Repeat the update steps for a number of epochs obtained from the collected trajectories.

---
